In [1]:
import ray

ray.init(logging_level="ERROR")


Python version:,3.10.15
Ray version:,2.37.0


In [2]:
import os
from PyPDF2 import PdfReader
import ray
import ray.data


pdf_dir = r"C:\Users\Vipul\RAG-PDF-QUERYING\data"

# Function to extract text from a PDF
def extract_text(filepath):
    reader = PdfReader(filepath)
    text = ''.join(page.extract_text() or '' for page in reader.pages)
    return text

# Prepare data
data = []
for filename in os.listdir(pdf_dir):
    if filename.lower().endswith('.pdf'):
        path = os.path.join(pdf_dir, filename)
        text = extract_text(path)
        data.append({"filename": filename, "content": text})


dataset = ray.data.from_items(data)

# (Optional) Show some data
print(dataset.take(2))




2024-10-18 18:43:19,823	INFO dataset.py:2416 -- Tip: Use `take_batch()` instead of `take() / show()` to return records in pandas or numpy batch format.
2024-10-18 18:43:19,832	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in C:\Users\Vipul\AppData\Local\Temp\ray\session_2024-10-18_18-42-51_240098_31748\logs\ray-data
2024-10-18 18:43:19,832	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> LimitOperator[limit=2]


Running 0: 0.00 row [00:00, ? row/s]

- limit=2 1: 0.00 row [00:00, ? row/s]

[{'filename': 'CleanBot_Robotic_Vacuum_Cleaner_FAQ.pdf', 'content': 'CleanBot Robotic Vacuum Cleaner FAQ \n1. Product Overview \nCleanBot offers a range of robotic vacuum cleaners to suit different cleaning needs. Our \ncurrent lineup includes: \n• CB-100 (Basic) \n• CB-200 (Smart Navigation) \n• CB-300 (Self-Emptying) \nEach model is designed to provide efficient and hassle-free cleaning. \n2. Technical Specifications \nCB-100 (Basic) \nBattery Life: 90 minutes \nSuction Power: 2000Pa \nDustbin Capacity: 0.5L \nNoise Level: 60dB \nWeight: 3.5kg \nCB-200 (Smart Navigation) \nBattery Life: 90 minutes \nSuction Power: 2000Pa \nDustbin Capacity: 0.5L \nNoise Level: 60dB \nWeight: 3.5kg \nCB-300 (Self-Emptying) \nBattery Life: 90 minutes \nSuction Power: 2000Pa \nDustbin Capacity: 0.5L Noise Level: 60dB \nWeight: 3.5kg \n3. Key Features \n• Efficient Cleaning: High suction power and multiple cleaning modes. \n• Smart Navigation: Advanced sensors for better navigation and cleaning. \n• Self

In [3]:
dataset.show()


2024-10-18 18:44:39,023	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in C:\Users\Vipul\AppData\Local\Temp\ray\session_2024-10-18_18-42-51_240098_31748\logs\ray-data
2024-10-18 18:44:39,023	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> LimitOperator[limit=20]


Running 0: 0.00 row [00:00, ? row/s]

- limit=20 1: 0.00 row [00:00, ? row/s]

{'filename': 'CleanBot_Robotic_Vacuum_Cleaner_FAQ.pdf', 'content': 'CleanBot Robotic Vacuum Cleaner FAQ \n1. Product Overview \nCleanBot offers a range of robotic vacuum cleaners to suit different cleaning needs. Our \ncurrent lineup includes: \n• CB-100 (Basic) \n• CB-200 (Smart Navigation) \n• CB-300 (Self-Emptying) \nEach model is designed to provide efficient and hassle-free cleaning. \n2. Technical Specifications \nCB-100 (Basic) \nBattery Life: 90 minutes \nSuction Power: 2000Pa \nDustbin Capacity: 0.5L \nNoise Level: 60dB \nWeight: 3.5kg \nCB-200 (Smart Navigation) \nBattery Life: 90 minutes \nSuction Power: 2000Pa \nDustbin Capacity: 0.5L \nNoise Level: 60dB \nWeight: 3.5kg \nCB-300 (Self-Emptying) \nBattery Life: 90 minutes \nSuction Power: 2000Pa \nDustbin Capacity: 0.5L Noise Level: 60dB \nWeight: 3.5kg \n3. Key Features \n• Efficient Cleaning: High suction power and multiple cleaning modes. \n• Smart Navigation: Advanced sensors for better navigation and cleaning. \n• Self-

In [6]:
import re
import ray
import ray.data

# Initialize Ray


# Define section headers
common_headers = [
    r"1\. Product Overview", r"2\. Technical Specifications", r"3\. Key Features",
    r"4\. Setup and Installation", r"5\. Usage Instructions", r"6\. Maintenance and Care",
    r"7\. Troubleshooting", r"8\. Warranty Information", r"9\. Customer Support"
]

techmobile_headers = [
    r"1\. Product Overview", r"2\. Technical Specifications", r"3\. Key Features",
    r"4\. Battery Life", r"5\. Camera", r"6\. Connectivity", r"7\. Troubleshooting"
]

# Function to clean page breaks
def clean_page_breaks(text):
    return re.sub(r'\n+', '\n', text)

# Function to split into sections
def split_sections(row):
    text = clean_page_breaks(row["content"])
    headers = techmobile_headers if row["filename"] == "TechMobile_Smartphone_FAQ.pdf" else common_headers
    sections = {}
    current_section = None
    current_content = []
    header_pattern = re.compile(rf"({'|'.join(headers)})", re.IGNORECASE)
    
    for line in text.split('\n'):
        match = header_pattern.match(line.strip())
        if match:
            if current_section:
                sections[current_section] = '\n'.join(current_content).strip()
            current_section = match.group(0)
            current_content = []
        elif current_section:
            current_content.append(line)
    
    if current_section:
        sections[current_section] = '\n'.join(current_content).strip()
    
    row["sections"] = sections
    return row



# Apply the split_sections function
split_dataset = dataset.map(split_sections)

# Show sections for 'TechMobile_Smartphone_FAQ.pdf' and one common PDF
techmobile = split_dataset.filter(lambda row: row["filename"] == "TechMobile_Smartphone_FAQ.pdf").take(1)
common_pdf = split_dataset.filter(lambda row: row["filename"] != "TechMobile_Smartphone_FAQ.pdf").take(1)

print("--- TechMobile_Smartphone_FAQ.pdf ---")
for section, content in techmobile[0]["sections"].items():
    print(f"--- {section} ---\n{content}\n")

print("--- Common PDF Example ---")
for section, content in common_pdf[0]["sections"].items():
    print(f"--- {section} ---\n{content}\n")

# Shutdown Ray when done
# ray.shutdown()


2024-10-18 18:54:19,874	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in C:\Users\Vipul\AppData\Local\Temp\ray\session_2024-10-18_18-42-51_240098_31748\logs\ray-data
2024-10-18 18:54:19,874	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[Map(split_sections)->Filter(<lambda>)] -> LimitOperator[limit=1]


Running 0: 0.00 row [00:00, ? row/s]

- Map(split_sections)->Filter(<lambda>) 1: 0.00 row [00:00, ? row/s]

- limit=1 2: 0.00 row [00:00, ? row/s]

2024-10-18 18:54:22,298	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in C:\Users\Vipul\AppData\Local\Temp\ray\session_2024-10-18_18-42-51_240098_31748\logs\ray-data
2024-10-18 18:54:22,298	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[Map(split_sections)->Filter(<lambda>)] -> LimitOperator[limit=1]


Running 0: 0.00 row [00:00, ? row/s]

- Map(split_sections)->Filter(<lambda>) 1: 0.00 row [00:00, ? row/s]

- limit=1 2: 0.00 row [00:00, ? row/s]

--- TechMobile_Smartphone_FAQ.pdf ---
--- 1. Product Overview ---
TechMobile offers a range of smartphones to suit various needs and preferences. Our 
current lineup includes: 
• TM-S100 (Standard) 
• TM-P200 (Pro) 
• TM-L150 (Lite) 
Each device is designed with cutting-edge technology to provide an exceptional user 
experience.

--- 2. Technical Specifications ---
TM-S100 (Standard) 
Processor: Snapdragon 888 
Memory: 8GB RAM 
Storage: 128GB 
Display: 6.5-inch AMOLED, 1080x2400 pixels 
Camera: 64MP main, 12MP ultra-wide, 5MP macro 
Battery Capacity: 4500mAh 
TM-P200 (Pro) 
Processor: Snapdragon 888 
Memory: 8GB RAM 
Storage: 128GB 
Display: 6.5-inch AMOLED, 1080x2400 pixels 
Camera: 64MP main, 12MP ultra-wide, 5MP macro 
Battery Capacity: 4500mAh 
TM-L150 (Lite) 
Processor: Snapdragon 888 Memory: 8GB RAM 
Storage: 128GB 
Display: 6.5-inch AMOLED, 1080x2400 pixels 
Camera: 64MP main, 12MP ultra-wide, 5MP macro 
Battery Capacity: 4500mAh

--- 3. Key Features ---
• AI-Enhanced Photograph